# Master TIDE - Conférences Python 2020

Francis Wolinski

&copy; 2020 Yotta Conseil


## Cartographies avec `plotly`

In [ ]:
# imports
import os

import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import cm
%matplotlib inline

# display options
pd.set_option("display.max_rows", 20)

### Dataset

In [ ]:
# load countries
if os.path.exists('GeoNames.htm') :
    var = pd.read_html('GeoNames.htm',
                       header=0,
                       keep_default_na=False,
                       encoding='latin-1')
else:
    var = pd.read_html('https://www.geonames.org/countries/',
                       header=0,
                       keep_default_na=False,
                       encoding='latin-1')
df = var[1]
df.shape

In [ ]:
df.head()

## Cartes avec `plotly`

Installation de `plotly` et de `colorlover` avec `conda`:

`plotly` est une librairie graphique développée par une société canadienne. An interactive, browser-based graphing library for Python. Voir aussi `dash` un framework pour produire des applications web en Python basé sur `flask`.

`colorlover` est une librairie d'échelle de couleurs utilisée.

Installation, ouvrir un terminal dans le dossier `anaconda3/condabin` :

- PC: `.\conda install -c plotly plotly`
- Mac: `./conda install -c plotly plotly`


- PC: `.\conda install -c conda-forge colorlover`
- Mac: `./conda install -c conda-forge colorlover`

In [ ]:
# imports
from plotly.offline import iplot, init_notebook_mode
import colorlover as cl

# init
init_notebook_mode(connected=True)

Avec `plotly` il est possible de définir des graphiques de manière déclarative, en définissant une structure de données et avec peu de lignes de code.

#### Carte avec `choropleth` (polygones)

In [ ]:
data = [{
        'type': 'choropleth',
        'locations': df['ISO-3166alpha3'],
        'z': df['Population'],
        'text': df['Country'],
        'colorscale': list(zip(np.linspace(0, 1, 9), cl.scales['9']['seq']['Reds'])),
        'autocolorscale': False,
        'reversescale': False,
        'marker': {
            'line': {
                'color': 'rgb(180,180,180)',
                'width': 0.5
            }},
        'colorbar':
            {'autotick': False,
            'title': 'Population'},
        }]

layout = {
    'title': '<br>Source:\
            <a href="https://www.geonames.org/countries/">\
            geonames</a>',
    'geo': {
        'showcountries': True,
        'showframe': False,
        'showcoastlines': False,
        'projection': {
            'type': 'natural earth'
        }
    }
}


fig = {'data': data, 'layout': layout}
iplot(fig, validate=False, filename='d3-world-map')

<div class="alert alert-success">
<b>Exercise 1</b>
<ul>
<li>Ajouter une colonne 'Densité' au 'DataFrame'.</li>
<li>Afficher la densité en utilisant la colormap 'Blues'.</li>
<li>Limiter les valeurs de la densité pour obtenir les nuances de couleur.</li>
<li>Choisir une projection parmi les suivantes : 'equirectangular', 'mercator', 'orthographic', 'natural earth', 'kavrayskiy7', 'miller', 'robinson', 'eckert4', 'azimuthal equal area', 'azimuthal equidistant', 'conic equal area', 'conic conformal', 'conic equidistant', 'gnomonic', 'stereographic', 'mollweide', 'hammer', 'transverse mercator'</li>
</ul>
</div>

#### Carte avec `scattergeo` (points)

In [ ]:
# cities5000.txt
villes = pd.read_csv('cities5000.txt',
                   sep='\t',
                   header=None,
                   names=['geonameid', 'name', 'asciiname', 'alternatenames', 'latitude', 'longitude', 'feature class', 'feature code', 'country code', 'cc2', 'admin1 code', 'admin2 code', 'admin3 code', 'admin4 code', 'population', 'elevation', 'dem', 'timezone', 'modification date'],
                   dtype={'admin1 code': str, 'admin2 code': str, 'admin3 code': str, 'admin4 code': str},
                   keep_default_na=False,
                   usecols=[2, 4, 5, 7, 8, 14])
villes.columns = ['asciiname', 'latitude', 'longitude', 'type', 'iso_2', 'population']
capitales = villes.loc[villes['type'] == 'PPLC']
capitales.head()

In [ ]:
# jointure pour obtenir latitude et longitude
df = pd.merge(df, capitales, left_on='ISO-3166alpha2', right_on='iso_2', how='left')
df.head()

In [ ]:
# affichage des pays où la densité est supérieure à 200 h/km²
df['Densité'] = df['Population'] / df['Area in km²']
df2 = df.loc[df['Densité']>200]

data = [{
        'type': 'scattergeo',
        'locations': df2['ISO-3166alpha3'],
        'text': df2['Country'] + ' (' + df2['Densité'].astype(int).astype(str) + ')',
        'z': df2['Densité'],
        'lon': df2['longitude'].values,
        'lat': df2['latitude'].values,
        'mode': 'markers',
        'marker': {
            'symbol': 'star',
            'color': 'navy',
            'size': 8
            },
        }]

layout = {
    'title': '<br>Source:\
            <a href="https://www.geonames.org/countries/">\
            geonames</a>',
    'geo': {
        'countrycolor': 'white',
        'showland': True,
        'landcolor': 'lightgrey',
        'showcountries': True,
        'showframe': False,
        'showcoastlines': False,
        'projection': {
            'type': 'natural earth'
        }
    }
}

fig = {'data': data, 'layout': layout}
iplot(fig, validate=False, filename='d3-world-map')

Il est possible de combiner des cartes `choropleth` et  `scattergeo` en partionnant les pays et en insérant autant de dictionnaires dans la liste data.

<div class="alert alert-warning">
<b>Pour plus d'information</b>
<ul>
    <li><a href="https://plot.ly/">https://plot.ly/</a></li>
</ul>
</div>

### Géocodage avec le module `geopy` et cartes avec le module `ipyleaflet` (démo)

`geopy` permet de géocoder des adresses physiques.

`ipyleaflet` fournit des cartes interactives pour Jupyter basé sur la librairie javascript leaflet.js et OpenStreetMap.

Installation, ouvrir un terminal dans le dossier `anaconda3/condabin` :

- PC: `.\conda install -c conda-forge geopy`
- Mac: `./conda install -c conda-forge geopy`


- PC: `.\conda install -c conda-forge ipyleaflet`
- Mac: `./conda install -c conda-forge ipyleaflet`


- PC: `.\jupyter nbextension enable --py --sys-prefix ipyleaflet`
- Mac: `./jupyter nbextension enable --py --sys-prefix ipyleaflet`

### Géocodage

Prior to display a position on a map, we will use the `geopy` module to get latitude and longitude from a real world address: <pre>15 Boulevard Diderot, 75012 Paris</pre>

Geocoding is the process of taking input text, such as an address or the name of a place, and returning a latitude/longitude location on the Earth's surface for that place.

The `geopy` library provides an interface to a list of geolocation service you might use, such as Google Maps, Bing Maps, or Nominatim.

In [ ]:
# si erreur Python ssl.SSLError
#import ssl
#import certifi
#ctx = ssl.create_default_context(cafile=certifi.where())
#import geopy
#geopy.geocoders.options.default_ssl_context = ctx

from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent='TIDE', timeout=7)
location = geolocator.geocode('90 rue de Tolbiac, 75013 Paris')
location.raw

In [ ]:
# latitude et longitude
lat, lon = location.raw['lat'], location.raw['lon']
lat, lon

<div class="alert alert-warning">
<b>Pour plus d'information</b>
<ul>
    <li><a href="https://geopy.readthedocs.io/en/stable/index.html">https://geopy.readthedocs.io/en/stable/index.html</a></li>
</ul>
</div>

### Carte standard

In [ ]:
# démo
from ipyleaflet import Map

# carte autour de 48.8272513, 2.3648174
m = Map(center=(lat, lon), zoom=18)
m

### Carte satellite

In [ ]:
# carte satellite
from datetime import datetime, timedelta
from ipyleaflet import basemaps, basemap_to_tiles

# carte satellite de la veille
# NASA's Global Imagery Browse Services
yesterday = datetime.now() - timedelta(days=1)
nasa = basemap_to_tiles(basemaps.NASAGIBS.ModisTerraTrueColorCR, yesterday.strftime('%Y-%m-%d'))
m = Map(layers=(nasa, ), center=(lat, lon), zoom=4)

m

### Autre carte

In [ ]:
# autre carte avec marqueurs

from ipyleaflet import Marker, LayerGroup

watercolor = basemap_to_tiles(basemaps.Stamen.Watercolor)
m = Map(layers=(watercolor, ), center=(50, 50), zoom=1)

# marqueurs pour les villes dont la population >= 3M
villes = villes.loc[villes['population'] >= 3e6]
layers = []
for i, row in villes.iterrows():
    marker = Marker(location=(row['latitude'], row['longitude']),
                    draggable=False,
                    title=row['asciiname'])
    layers.append(marker)
layer_group = LayerGroup(layers=layers)
m.add_layer(layer_group)

m

<div class="alert alert-warning">
<b>Pour plus d'information</b>
<ul>
    <li><a href="https://ipyleaflet.readthedocs.io/en/latest/index.html">https://ipyleaflet.readthedocs.io/en/latest/index.html</a></li>
</ul>
</div>